<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.8/546.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-30 20:46:15
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: 72.64 K (0.54%)
Current PnL: -13.54 L (-9.43%)
CY Booked + Current PnL: -6.80 L (-4.74%)
-------------------
Total profit:  4.41 L
Total loss:  -17.95 L
-------------------
Total Booked + Current PnL: 20.18 L (17.32%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.94%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.96 L (52.77%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.74%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,82.29,74398.0,39949.0,0.0,2.58,115.97,0.00,115.97,NaN,inf,0.55,13.77,AR,ATH,ELECTRICAL
16,BLUSPRING,226.45,84.55,NaN,NaN,5.21,16631.0,-32962.0,1886.0,-3.24,-66.46,11.34,-62.66,NaN,-17.48,0.12,11.53,XY24,ATH,MISC
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,42952.0,1697.0,9110.0,-2.83,4.11,21.21,26.19,1.0,0.19,0.32,0.00,XY24,ATH,IT
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,44.79,102964.0,24602.0,9432.0,0.69,31.40,9.16,43.43,77.0,2.61,0.76,38.38,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,60.94,174643.0,33803.0,10688.0,3.05,24.00,6.12,31.59,89.0,3.16,1.28,51.70,XY24,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,LALPATHLAB,2804.19,3545.00,-33.15,H-SC,39.58,153796.0,-434.0,41171.0,0.99,-0.28,26.77,26.42,125.0,-0.01,1.13,21.28,X40N,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2337.55,-12.78,H-SC,73.96,163560.0,-1110.0,70200.0,0.71,-0.67,42.92,41.95,121.0,-0.02,1.20,6.74,X40N,ATH,AC


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SATIN,199.77,274.00,-15.90,H-SC,83.33,166446.0,-29528.0,102348.0,7.87,-15.07,61.49,37.16,130.0,-0.29,1.22,24.30,XY24,NTT,FINANCE
4,ALKYLAMINE,2347.98,4546.37,9.45,X-SC,100.00,101824.0,861.0,93668.0,4.69,0.85,91.99,93.63,64.0,0.01,0.75,51.66,SR,ATH,CHEMICALS
54,KPIGREEN,497.21,732.16,26.70,H-SC,66.67,129692.0,4395.0,54808.0,3.95,3.51,42.26,47.25,139.0,0.08,0.95,62.25,MH,ATH,POWER
29,DIXON,14367.00,18940.15,-34.88,H-MC,46.88,119864.0,4928.0,31656.0,3.52,4.29,26.41,31.83,99.0,0.16,0.88,37.38,X40N,ATH,IT
88,UNIONBANK,123.87,163.00,0.15,M-LC,60.94,174643.0,33803.0,10688.0,3.05,24.00,6.12,31.59,89.0,3.16,1.28,51.70,XY24,NTT,BANKS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,5.21,16631.0,-32962.0,1886.0,-3.24,-66.46,11.34,-62.66,NaN,-17.48,0.12,11.53,XY24,ATH,MISC
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,42952.0,1697.0,9110.0,-2.83,4.11,21.21,26.19,1.0,0.19,0.32,0.00,XY24,ATH,IT
90,VALIANTORG,512.64,838.00,-350.78,H-SC,6.25,106535.0,-26751.0,111340.0,-2.75,-20.07,104.51,63.47,148.0,-0.24,0.78,78.58,XR,NTT,CHEMICALS
38,HEROMOTOCO,4311.81,6039.03,-7.62,H-MC,18.75,148298.0,-2615.0,63071.0,-1.93,-1.73,42.53,40.06,102.0,-0.04,1.09,21.30,AR,ATH,AUTO
72,SFL,1016.35,1287.00,41.99,H-SC,95.83,187476.0,-74742.0,144563.0,-1.39,-28.50,77.11,26.63,144.0,-0.52,1.38,16.56,XY24,NTT,MISC


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,52.86,M-SC,92.71,132381.0,-10098.0,10723.0,0.49,-7.09,8.10,0.44,243.0,-0.94,0.97,56.25,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-213.83,H-SC,96.88,82124.0,-13192.0,13230.0,0.95,-13.84,16.11,0.04,154.0,-1.00,0.60,61.98,OX40N,NTT,MISC
81,TATAELXSI,7332.28,7332.0,-15.80,X-MC,25.00,88340.0,-14312.0,14311.0,-0.55,-13.94,16.20,-0.00,58.0,-1.00,0.65,34.29,OX40N,NTT,IT
53,KANSAINER,299.63,340.0,-67.47,H-SC,37.50,226935.0,-42732.0,79064.0,1.63,-15.85,34.84,13.47,142.0,-0.54,1.66,14.78,XY24,NTT,PAINTS
46,INDIGOPNTS,1407.73,1408.0,119.87,H-SC,77.08,144113.0,-30446.0,30480.0,0.09,-17.44,21.15,0.02,127.0,-1.00,1.06,24.97,OX40N,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,42952.0,1697.0,9110.0,-2.83,4.11,21.21,26.19,1.0,0.19,0.32,0.00,XY24,ATH,IT
83,TCS,3794.03,4998.00,-17.10,X-LC,41.67,270036.0,-25898.0,119815.0,0.61,-8.75,44.37,31.73,1.0,-0.22,1.98,8.09,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.62,X-LC,51.04,286722.0,25121.0,120509.0,-0.39,9.60,42.03,55.67,2.0,0.21,2.10,16.29,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-18.00,X-LC,22.92,256995.0,-13352.0,47981.0,-0.53,-4.94,18.67,12.81,6.0,-0.28,1.89,7.48,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-9.04,X-LC,52.08,288440.0,23014.0,33892.0,0.27,8.67,11.75,21.44,9.0,0.68,2.12,15.40,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
94,WHIRLPOOL,1167.49,2270.0,-37.55,M-SC,91.67,206006.0,34385.0,127683.0,-0.10,20.04,61.98,94.43,198.0,0.27,1.51,52.05,XR,NTT,DURABLES
88,UNIONBANK,123.87,163.0,0.15,M-LC,60.94,174643.0,33803.0,10688.0,3.05,24.00,6.12,31.59,89.0,3.16,1.28,51.70,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.40,M-LC,44.79,102964.0,24602.0,9432.0,0.69,31.40,9.16,43.43,77.0,2.61,0.76,38.38,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,82.29,74398.0,39949.0,0.0,2.58,115.97,0.00,115.97,NaN,inf,0.55,13.77,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,89.46,M-MC,47.92,227287.0,2325.0,165329.0,0.22,1.03,72.74,74.53,185.0,0.01,1.67,31.96,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,26.70,H-SC,66.67,129692.0,4395.0,54808.0,3.95,3.51,42.26,47.25,139.0,0.08,0.95,62.25,MH,ATH,POWER
20,CAMS,3643.00,5250.99,6.98,H-SC,81.25,120126.0,18122.0,26896.0,-0.06,17.77,22.39,44.14,124.0,0.67,0.88,36.89,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,44.79,102964.0,24602.0,9432.0,0.69,31.40,9.16,43.43,77.0,2.61,0.76,38.38,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,60.94,174643.0,33803.0,10688.0,3.05,24.00,6.12,31.59,89.0,3.16,1.28,51.70,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,82.29,74398.0,39949.0,0.0,2.58,115.97,0.00,115.97,NaN,inf,0.55,13.77,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-37.55,M-SC,91.67,206006.0,34385.0,127683.0,-0.10,20.04,61.98,94.43,198.0,0.27,1.51,52.05,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-3.77,M-LC,72.92,164908.0,13963.0,95498.0,0.35,9.25,57.91,72.51,69.0,0.15,1.21,13.59,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-47.14,H-SC,89.58,137694.0,14358.0,119408.0,-0.39,11.64,86.72,108.45,122.0,0.12,1.01,36.60,AR,ATH,MISC
54,KPIGREEN,497.21,732.16,26.70,H-SC,66.67,129692.0,4395.0,54808.0,3.95,3.51,42.26,47.25,139.0,0.08,0.95,62.25,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,97.92,71904.0,-41645.0,81697.0,1.82,-36.68,113.62,35.27,261.0,-0.51,0.53,75.99,XR,NTT,HOTELS
32,ENRIN,1377.95,2975.90,NaN,NaN,82.29,74398.0,39949.0,0.0,2.58,115.97,0.00,115.97,NaN,inf,0.55,13.77,AR,ATH,ELECTRICAL
58,MASFIN,326.60,402.39,-18.70,H-SC,59.38,92415.0,-5565.0,28297.0,-1.06,-5.68,30.62,23.21,137.0,-0.20,0.68,34.03,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,9.45,X-SC,100.00,101824.0,861.0,93668.0,4.69,0.85,91.99,93.63,64.0,0.01,0.75,51.66,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,44.79,102964.0,24602.0,9432.0,0.69,31.40,9.16,43.43,77.0,2.61,0.76,38.38,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,2347.98,4546.37,9.45,X-SC,100.00,101824.0,861.0,93668.0,4.69,0.85,91.99,93.63,64.0,0.01,0.75,51.66,SR,ATH,CHEMICALS
52,JPPOWER,18.73,26.20,-30.97,L-SC,98.96,138586.0,-3256.0,59828.0,1.11,-2.30,43.17,39.88,259.0,-0.05,1.02,40.45,XY24,NTT,POWER
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,97.92,71904.0,-41645.0,81697.0,1.82,-36.68,113.62,35.27,261.0,-0.51,0.53,75.99,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-213.83,H-SC,96.88,82124.0,-13192.0,13230.0,0.95,-13.84,16.11,0.04,154.0,-1.00,0.60,61.98,OX40N,NTT,MISC
72,SFL,1016.35,1287.00,41.99,H-SC,95.83,187476.0,-74742.0,144563.0,-1.39,-28.50,77.11,26.63,144.0,-0.52,1.38,16.56,XY24,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.98
1,25,42.01
2,50,70.40


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
SC,43.50
LC,34.12
MC,21.42


In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Criteria,
XY24,31.54
X40,12.21
XY25,12.09
X40N,11.76
XR,10.16
AR,8.31
OX40N,8.14
MH,1.70
X5K,1.47


In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
H-SC,20.63
M-SC,16.03
H-LC,14.16
X-LC,12.23
H-MC,9.48
X-MC,7.54
M-LC,6.67
X-SC,4.38
M-MC,4.04


In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.17,-1.24,35.09
IT,12.11,-10.12,64.22
FINANCE,7.97,-11.18,51.42
BANKS,7.96,-4.91,47.72
MISC,6.66,-14.63,67.70
PAINTS,5.65,-13.33,35.67
ELECTRICAL,5.54,2.13,43.57
HEALTHCARE,4.48,-5.56,36.40
AUTO,4.34,-14.41,64.72


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2704976.0,27
XR,1074913.0,13
AR,1011933.0,10
X40,654620.0,9
X40N,562654.0,11
XY25,471938.0,8
OX40N,320508.0,11
SR,174554.0,2
MH,83823.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1970749.0,20
M-SC,1760383.0,21
H-LC,611040.0,12
H-MC,541337.0,8
X-LC,508125.0,7
X-MC,426071.0,6
M-MC,420386.0,3
X-SC,322823.0,5
M-LC,263782.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      850750.0      6
M-SC       XY24      811848.0      7
H-SC       AR        519056.0      3
M-SC       XR        331921.0      4
M-MC       XY24      310505.0      2
H-MC       XY24      297086.0      3
X-LC       X40       240324.0      2
H-SC       X40N      233124.0      4
           XR        218823.0      3
H-LC       X40N      200273.0      4
M-SC       AR        180129.0      2
           OX40N     172818.0      5
H-LC       X40       160565.0      3
M-LC       XY24      156980.0      3
X-MC       X40       151271.0      2
L-SC       XR        143335.0      2
X-SC       XR        122564.0      1
X-LC       AR        117957.0      1
M-SC       XY25      110177.0      1
M-MC       XR        109881.0      1
X-LC       XY25      109245.0      3
X-MC       XY25      106081.0      1
           XY24       97406.0      1
M-LC       XR         95498.0      1
H-SC       OX40N      94188.0      3
X-SC       SR         93668.0      1
H-MC       XY25       90653.0      1
M-SC       SR         80886.0      1
           X40        72604.0      1
H-LC       AR         72280.0      1
           X200       68870.0      1
X-SC       XY24       67400.0      1
H-LC       X5K        66875.0      2
H-MC       AR         63071.0      1
L-SC       XY24       59828.0      1
           AR         59440.0      1
X-MC       X40N       57002.0      1
H-SC       MH         54808.0      1
L-MC       XR         52891.0      1
L-LC       XY25       44478.0      1
H-LC       XY24       42177.0      1
X-LC       X40N       40599.0      1
X-SC       OX40N      39191.0      2
H-MC       X40N       31656.0      1
           X40        29856.0      1
           MH         29015.0      1
X-MC       OX40N      14311.0      1
M-LC       XY25       11304.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 46.0 seconds
